In [1]:
!pip install torch pandas numpy scikit-learn biopython

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from Bio import SeqIO # 这是Biopython中用于读取FASTA文件的核心模块

print("所有库已成功导入！")
print(f"PyTorch版本: {torch.__version__}")

所有库已成功导入！
PyTorch版本: 2.8.0+cpu


In [3]:
# 数据处理
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 深度学习
import torch
from torch.utils.data import Dataset, DataLoader

# 生物学数据处理（核心！）
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

print("所有库导入成功！")

所有库导入成功！


In [4]:
# 定义两个FASTA文件的完整路径列表
fasta_file_paths = [
    r"C:\Users\奶罗咪\Desktop\ncbi_dataset\ncbi_dataset\data\GCF_000001405.40\protein.faa",  # 主文件
    r"C:\Users\奶罗咪\Desktop\ncbi_dataset\ncbi_dataset\data\GCA_000001405.29\protein.faa"   # 补充文件
]

print("将要处理以下文件：")
for i, path in enumerate(fasta_file_paths, 1):
    print(f"{i}. {path}")

将要处理以下文件：
1. C:\Users\奶罗咪\Desktop\ncbi_dataset\ncbi_dataset\data\GCF_000001405.40\protein.faa
2. C:\Users\奶罗咪\Desktop\ncbi_dataset\ncbi_dataset\data\GCA_000001405.29\protein.faa


In [6]:
def parse_fasta_file(file_path):
    """
    读取一个FASTA文件，返回包含蛋白质信息的DataFrame
    """
    protein_ids = []
    protein_descriptions = []
    protein_sequences = []
    
    print(f"开始读取文件: {file_path}")
    
    try:
        for record in SeqIO.parse(file_path, "fasta"):
            protein_ids.append(record.id)
            protein_descriptions.append(record.description)
            protein_sequences.append(str(record.seq))
            
        # 创建DataFrame
        df = pd.DataFrame({
            'protein_id': protein_ids,
            'description': protein_descriptions,
            'sequence': protein_sequences,
            'sequence_length': [len(seq) for seq in protein_sequences],
            'source_file': file_path.split('\\')[-2]  # 记录数据来源，如 GCF_000001405.40
        })
        
        print(f"✅ 成功读取 {len(df)} 条序列")
        return df
        
    except Exception as e:
        print(f"❌ 读取文件时出错: {e}")
        return None

In [7]:
# 创建一个空列表来存储每个文件的数据
all_dfs = []

# 循环处理每个FASTA文件
for file_path in fasta_file_paths:
    df = parse_fasta_file(file_path)
    if df is not None:
        all_dfs.append(df)
    print("-" * 50)

# 检查是否成功读取了文件
if not all_dfs:
    print("没有成功读取任何文件，请检查路径是否正确。")
else:
    # 将所有DataFrame合并成一个
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"合并后总序列数: {len(combined_df)}")
    

开始读取文件: C:\Users\奶罗咪\Desktop\ncbi_dataset\ncbi_dataset\data\GCF_000001405.40\protein.faa
✅ 成功读取 136807 条序列
--------------------------------------------------
开始读取文件: C:\Users\奶罗咪\Desktop\ncbi_dataset\ncbi_dataset\data\GCA_000001405.29\protein.faa
✅ 成功读取 13 条序列
--------------------------------------------------
合并后总序列数: 136820


In [8]:
# 查看合并后的数据基本信息
print("合并后的数据预览：")
display(combined_df.head())
print(f"\n合并后数据形状: {combined_df.shape}")

# 检查是否有重复的蛋白质ID
duplicate_ids = combined_df['protein_id'].duplicated().sum()
print(f"重复的蛋白质ID数量: {duplicate_ids}")

# 基于蛋白质序列去重（更严格，因为不同ID可能有相同序列）
duplicate_sequences = combined_df['sequence'].duplicated().sum()
print(f"重复的蛋白质序列数量: {duplicate_sequences}")

# 去除重复序列，保留第一个出现的
final_df = combined_df.drop_duplicates(subset=['sequence'], keep='first')
print(f"去重后的唯一序列数: {len(final_df)}")

# 查看数据来源分布
print("\n数据来源分布：")
print(final_df['source_file'].value_counts())

合并后的数据预览：


,protein_id,description,sequence,sequence_length,source_file
0,NP_000005.3,NP_000005.3 alpha-2-macroglobulin isoform a pr...,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,1474,GCF_000001405.40
1,NP_000006.2,NP_000006.2 arylamine N-acetyltransferase 2 [H...,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...,290,GCF_000001405.40
2,NP_000007.1,NP_000007.1 medium-chain specific acyl-CoA deh...,MAAGFGRCCRVLRSISRFHWRSQHTKANRQREPGLGFSFEFTEQQK...,421,GCF_000001405.40
3,NP_000008.1,NP_000008.1 short-chain specific acyl-CoA dehy...,MAAALLARASGPARRALCPRAWRQLHTIYQSVELPETHQMLLQTCR...,412,GCF_000001405.40
4,NP_000009.1,NP_000009.1 very long-chain specific acyl-CoA ...,MQAARMAASLGRQLLRLGGGSSRLTALLGQPRPGPARRPYAGGAAQ...,655,GCF_000001405.40



合并后数据形状: (136820, 5)
重复的蛋白质ID数量: 0
重复的蛋白质序列数量: 46447
去重后的唯一序列数: 90373

数据来源分布：
source_file
GCF_000001405.40    90373
Name: count, dtype: int64


In [9]:
# 保存处理好的数据
output_path = r"C:\Users\奶罗咪\Desktop\ncbi_dataset\processed_protein_data_combined.csv"
final_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"处理好的数据已保存到: {output_path}")

# 查看最终数据集的信息
print("\n最终数据集信息：")
print(f"总蛋白质序列数: {len(final_df)}")
print(f"序列长度范围: {final_df['sequence_length'].min()} 到 {final_df['sequence_length'].max()} 个氨基酸")
print(f"平均序列长度: {final_df['sequence_length'].mean():.1f} 个氨基酸")

处理好的数据已保存到: C:\Users\奶罗咪\Desktop\ncbi_dataset\processed_protein_data_combined.csv

最终数据集信息：
总蛋白质序列数: 90373
序列长度范围: 12 到 35991 个氨基酸
平均序列长度: 715.7 个氨基酸


In [12]:
# --- Day 2 第一步：导入库和加载数据 ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示（确保图表能正常显示中文）
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 加载Day 1处理好的数据
print("正在加载Day 1处理好的数据...")
df = pd.read_csv(r"C:\Users\奶罗咪\Desktop\ncbi_dataset\processed_protein_data_combined.csv")
print(f" 数据加载成功！数据集形状: {df.shape}")

# 显示前几行数据，确认加载正确
print("\n 数据预览:")
display(df.head(3))

正在加载Day 1处理好的数据...
 数据加载成功！数据集形状: (90373, 5)

 数据预览:


,protein_id,description,sequence,sequence_length,source_file
0,NP_000005.3,NP_000005.3 alpha-2-macroglobulin isoform a pr...,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,1474,GCF_000001405.40
1,NP_000006.2,NP_000006.2 arylamine N-acetyltransferase 2 [H...,MDIEAYFERIGYKNSRNKLDLETLTDILEHQIRAVPFENLNMHCGQ...,290,GCF_000001405.40
2,NP_000007.1,NP_000007.1 medium-chain specific acyl-CoA deh...,MAAGFGRCCRVLRSISRFHWRSQHTKANRQREPGLGFSFEFTEQQK...,421,GCF_000001405.40


In [14]:
# --- Day 2 第二步：创建分类标签 ---

# 定义酶相关的关键词
enzyme_keywords = [
    'kinase', 'polymerase', 'transferase', 'hydrolase', 
    'oxidoreductase', 'lyase', 'isomerase', 'ligase', 
    'reductase', 'synthase', 'enzyme', 'dehydrogenase'
]

def is_enzyme(description):
    """根据蛋白质描述判断是否为酶"""
    if pd.isna(description):
        return False
    description_lower = str(description).lower()
    return any(keyword in description_lower for keyword in enzyme_keywords)

# 添加酶/非酶标签列
print("正在创建酶/非酶分类标签...")
df['is_enzyme'] = df['description'].apply(is_enzyme).astype(int)

# 查看标签分布
label_counts = df['is_enzyme'].value_counts()
print("\n 酶/非酶分类统计：")
print(f"非酶 (0): {label_counts[0]} 条序列")
print(f"酶 (1): {label_counts[1]} 条序列")
print(f"酶的比例: {df['is_enzyme'].mean():.2%}")

# 检查数据平衡性
if label_counts.min() / label_counts.sum() < 0.3:
    print("  注意：类别存在不平衡，将在训练时使用加权损失函数")
else:
    print(" 类别分布相对平衡")

正在创建酶/非酶分类标签...

 酶/非酶分类统计：
非酶 (0): 76370 条序列
酶 (1): 14003 条序列
酶的比例: 15.49%
  注意：类别存在不平衡，将在训练时使用加权损失函数


In [15]:
# --- Day 2 第三步：创建词汇表和标记化 ---

print("\n正在创建氨基酸词汇表...")

# 获取所有唯一的氨基酸字符
all_sequences = ''.join(df['sequence'].tolist())
unique_aa = sorted(set(all_sequences))

# 创建氨基酸到ID的映射字典
vocab = {aa: idx+1 for idx, aa in enumerate(unique_aa)}  # 从1开始编号
vocab['<PAD>'] = 0    # 填充标记
vocab['<UNK>'] = len(vocab)  # 未知氨基酸标记

print(f" 词汇表创建完成！共 {len(vocab)} 个字符")
print("示例映射:", list(vocab.items())[:8])

# 设置最大序列长度（基于您之前的数据：平均715，最大35991，我们取一个合理的值）
MAX_SEQ_LENGTH = 1500  # 这个值可以覆盖大部分序列
print(f"\n设置最大序列长度: {MAX_SEQ_LENGTH}")

# 将氨基酸序列转换为数字ID序列
def sequence_to_ids(sequence, vocab, max_len=None):
    """将蛋白质序列转换为数字ID序列"""
    ids = [vocab.get(aa, vocab['<UNK>']) for aa in sequence]
    if max_len:
        ids = ids[:max_len]  # 截断长序列
    return ids

print("正在转换序列为数字ID...")
df['token_ids'] = df['sequence'].apply(lambda x: sequence_to_ids(x, vocab, MAX_SEQ_LENGTH))
df['seq_len'] = df['token_ids'].apply(len)

print(" 序列标记化完成！")
print(f"截断后的序列长度统计:")
print(df['seq_len'].describe())


正在创建氨基酸词汇表...
 词汇表创建完成！共 24 个字符
示例映射: [('A', 1), ('C', 2), ('D', 3), ('E', 4), ('F', 5), ('G', 6), ('H', 7), ('I', 8)]

设置最大序列长度: 1500
正在转换序列为数字ID...
 序列标记化完成！
截断后的序列长度统计:
count    90373.000000
mean       636.772144
std        412.675862
min         12.000000
25%        315.000000
50%        520.000000
75%        870.000000
max       1500.000000
Name: seq_len, dtype: float64


In [4]:
# --- Day 2 步骤4: 创建PyTorch数据加载器 ---
# 首先导入必要的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

print("\n正在划分数据集并创建数据加载器...")

# 重新加载数据（确保df变量存在）
df = pd.read_csv(r"C:\Users\奶罗咪\Desktop\ncbi_dataset\processed_protein_data_combined.csv")
print(f"数据加载成功！形状: {df.shape}")

# 重新创建酶标签（确保is_enzyme列存在）
enzyme_keywords = [
    'kinase', 'polymerase', 'transferase', 'hydrolase', 'oxidoreductase',
    'lyase', 'isomerase', 'ligase', 'reductase', 'synthase', 'enzyme', 'dehydrogenase'
]

def is_enzyme(description):
    if pd.isna(description):
        return False
    description_lower = str(description).lower()
    return any(keyword in description_lower for keyword in enzyme_keywords)

df['is_enzyme'] = df['description'].apply(is_enzyme).astype(int)

# 重新进行标记化（确保token_ids列存在）
vocab = {}  # 这里需要您的词汇表，或者重新创建
# 如果您有保存的词汇表，可以加载：
# vocab_df = pd.read_csv(r"C:\Users\奶罗咪\Desktop\ncbi_dataset\amino_acid_vocab.csv")
# vocab = dict(zip(vocab_df['amino_acid'], vocab_df['token_id']))

# 或者重新创建词汇表
all_sequences = ''.join(df['sequence'].tolist())
unique_aa = sorted(set(all_sequences))
vocab = {aa: idx+1 for idx, aa in enumerate(unique_aa)}
vocab['<PAD>'] = 0
vocab['<UNK>'] = len(vocab)

MAX_SEQ_LENGTH = 1500
def sequence_to_ids(sequence, vocab, max_len=None):
    ids = [vocab.get(aa, vocab['<UNK>']) for aa in sequence]
    if max_len:
        ids = ids[:max_len]
    return ids

df['token_ids'] = df['sequence'].apply(lambda x: sequence_to_ids(x, vocab, MAX_SEQ_LENGTH))

# 现在进行数据划分
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['is_enzyme'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['is_enzyme'])

print(f"训练集: {len(train_df)} 条序列")
print(f"验证集: {len(val_df)} 条序列")
print(f"测试集: {len(test_df)} 条序列")

# 创建自定义数据集类
class ProteinDataset(Dataset):
    def __init__(self, sequences, labels, max_len=MAX_SEQ_LENGTH):
        self.sequences = sequences
        self.labels = labels
        self.max_len = max_len
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        label = self.labels[idx]
        
        # 填充序列到固定长度
        padded_seq = seq + [0] * (self.max_len - len(seq))
        padded_seq = padded_seq[:self.max_len]
        
        return torch.tensor(padded_seq, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# 创建数据集实例
train_dataset = ProteinDataset(train_df['token_ids'].tolist(), train_df['is_enzyme'].tolist())
val_dataset = ProteinDataset(val_df['token_ids'].tolist(), val_df['is_enzyme'].tolist())
test_dataset = ProteinDataset(test_df['token_ids'].tolist(), test_df['is_enzyme'].tolist())

# 创建数据加载器
BATCH_SIZE = 4  # 从 32 改为 4，解决内存不足问题

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f" 数据加载器创建完成！批次大小: {BATCH_SIZE} (已调整为较小值以避免内存不足)")


正在划分数据集并创建数据加载器...
数据加载成功！形状: (90373, 5)
训练集: 63261 条序列
验证集: 13556 条序列
测试集: 13556 条序列
 数据加载器创建完成！批次大小: 4 (已调整为较小值以避免内存不足)


In [5]:
# --- Day 2 步骤5: 构建Transformer模型 (修正版) ---
# 首先导入所有必要的库
import torch
import torch.nn as nn
import math
from torch.nn import TransformerEncoder, TransformerEncoderLayer

print("\n正在构建Transformer模型...")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class ProteinTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, num_layers=4, num_classes=2):
        super().__init__()
        # 添加这一行：保存d_model为类属性
        self.d_model = d_model
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, MAX_SEQ_LENGTH)
        
        # Transformer编码器
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=512, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        # 分类器
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, src):
        # src形状: [batch_size, seq_len]
        src = self.embedding(src) * math.sqrt(self.d_model)  # [batch_size, seq_len, d_model]
        src = src.transpose(0, 1)  # [seq_len, batch_size, d_model]
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)  # [seq_len, batch_size, d_model]
        output = output.mean(dim=0)  # 全局平均池化 [batch_size, d_model]
        return self.classifier(output)  # [batch_size, num_classes]

# 创建模型实例
model = ProteinTransformer(vocab_size=len(vocab))
print("✅ Transformer模型构建完成！")
print(f"模型参数量: {sum(p.numel() for p in model.parameters()):,}")


正在构建Transformer模型...
✅ Transformer模型构建完成！
模型参数量: 804,546


C:\Users\奶罗咪\AppData\Roaming\Python\Python313\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# --- Day 2 步骤6: 训练模型 ---

# 1. 重新创建模型实例（使用修正后的类）
model = ProteinTransformer(vocab_size=len(vocab))
print("✅ Transformer模型构建完成！")
print(f"模型参数量: {sum(p.numel() for p in model.parameters()):,}")

# 2. 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")
model = model.to(device)

# 3. 计算类别权重（解决label_counts未定义的问题）
# 首先需要计算标签的分布
print("正在计算类别权重...")

# 方法1：如果您有df变量，可以直接计算
try:
    label_counts = df['is_enzyme'].value_counts()
    print(f"标签分布: 非酶={label_counts[0]}, 酶={label_counts[1]}")
except:
    # 方法2：如果没有df，从数据加载器中统计
    enzyme_count = 0
    non_enzyme_count = 0
    for data, target in train_loader:
        enzyme_count += (target == 1).sum().item()
        non_enzyme_count += (target == 0).sum().item()
    label_counts = {0: non_enzyme_count, 1: enzyme_count}
    print(f"从训练集统计的标签分布: 非酶={non_enzyme_count}, 酶={enzyme_count}")

# 设置损失函数和优化器（处理类别不平衡）
class_weights = torch.tensor([1.0, label_counts[0]/label_counts[1]], dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"使用类别权重: {class_weights.cpu().numpy()}")

# 4. 训练循环
num_epochs = 5  # 先训练5个epoch看效果
train_losses, val_losses, val_accuracies = [], [], []

print("\n开始训练模型...")
for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # 验证阶段
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100. * correct / total
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs} - '
          f'Train Loss: {avg_train_loss:.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, '
          f'Val Acc: {val_accuracy:.2f}%')

print(" 模型训练完成！")

C:\Users\奶罗咪\AppData\Roaming\Python\Python313\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


✅ Transformer模型构建完成！
模型参数量: 804,546
使用设备: cpu
正在计算类别权重...
标签分布: 非酶=76370, 酶=14003
使用类别权重: [1.       5.453831]

开始训练模型...


In [10]:
# --- 紧急保存 ---
import os
import pickle

print("尝试紧急保存...")

# 1. 首先检查最重要的变量是否存在
try:
    print(f"vocab 类型: {type(vocab)}, 大小: {len(vocab)}")
    vocab_exists = True
except:
    print("vocab 不存在")
    vocab_exists = False

try:
    print(f"BATCH_SIZE: {BATCH_SIZE}")
    batch_size_exists = True  
except:
    print("BATCH_SIZE 不存在")
    batch_size_exists = False

# 2. 尝试保存到桌面（避免权限问题）
desktop_path = r"C:\Users\奶罗咪\Desktop"

if vocab_exists:
    try:
        vocab_file = os.path.join(desktop_path, "protein_vocab.pkl")
        with open(vocab_file, 'wb') as f:
            pickle.dump(vocab, f)
        print(f"✅ 词汇表已保存到桌面: {vocab_file}")
    except Exception as e:
        print(f"❌ 保存失败: {e}")

# 3. 保存配置
config = {
    'class_weights': [1.0, 5.453831],
    'learning_rate': 0.001,
    'BATCH_SIZE': BATCH_SIZE if batch_size_exists else 4,
    'MAX_SEQ_LENGTH': 1500
}

try:
    config_file = os.path.join(desktop_path, "training_config.pkl")
    with open(config_file, 'wb') as f:
        pickle.dump(config, f)
    print(f"✅ 配置已保存到桌面: {config_file}")
except Exception as e:
    print(f"❌ 配置保存失败: {e}")

# 4. 检查桌面文件
print("\n桌面文件列表:")
for file in os.listdir(desktop_path):
    if file.endswith('.pkl'):
        print(f"📄 {file}")

print("紧急保存完成！")

尝试紧急保存...
vocab 类型: <class 'dict'>, 大小: 24
BATCH_SIZE: 4
✅ 词汇表已保存到桌面: C:\Users\奶罗咪\Desktop\protein_vocab.pkl
✅ 配置已保存到桌面: C:\Users\奶罗咪\Desktop\training_config.pkl

桌面文件列表:
📄 protein_vocab.pkl
📄 training_config.pkl
紧急保存完成！
